In [2]:
!pip install beautifulsoup4
!pip install lxml
!conda install -c conda-forge folium=0.5.0 --yes
!pip install geopy
import pandas as pd
import numpy as np
import requests 
import random
from geopy.geocoders import Nominatim 
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: ...working... done

# All requested packages already installed.



##First part of the work begins##

In [3]:
#Scatch the data fro wilipedia by BeautifulSoup
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
data=BeautifulSoup(source,'lxml')
from IPython.display import display_html
tab = str(data.table)
dff = pd.read_html(tab)
df=dff[0]
df.head(12)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [4]:
#Remove the rows containing "Not assigned" in the 'Borough' column
dff = df.drop(df[(df['Borough']=='Not assigned')].index)
dff.rename(columns={'Postal Code':'Postcode'},inplace=True)
dfff = dff.groupby(['Postcode','Borough'], sort=False).agg(','.join)
dfff.head(12)

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"
M3B,North York,Don Mills
M4B,East York,"Parkview Hill, Woodbine Gardens"


In [5]:
#If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
dfff.reset_index(inplace=True)
dfff['Neighbourhood'] = np.where(dfff['Neighbourhood'] == 'Not assigned',dfff['Borough'], dfff['Neighbourhood'])
dfff.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


First part of the work is done and a panda frame result is shown.

##Second part of the work begins##

In [6]:
#Get the location data and incoporate
location = pd.read_csv('https://cocl.us/Geospatial_data')
location.rename(columns={'Postal Code':'Postcode'},inplace=True)
newdf = pd.merge(dfff,location,on='Postcode')
newdf.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


##The second part of the work is done here by showing the postcode with latitude and longitude

##Third part of the work begins##

In [24]:
# Screen out Toronto only
Torontodf = newdf[newdf['Borough'].str.contains('Toronto',regex=False)]
Torontodf.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [25]:
import folium
map_toronto = folium.Map(location=[43.65,-79.35],zoom_start=12)

for lat,lng,borough,neighbourhood in zip(Torontodf['Latitude'],Torontodf['Longitude'],Torontodf['Borough'],Torontodf['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='green',
    fill=True,
    fill_color='#3386cc',
    fill_opacity=0.6,
    parse_html=False).add_to(map_toronto)
map_toronto

It looks many green labeled dots in downtown, Toronto. Let's see how the clustering will be! I guess there will be a large number near Downtown, Toronto

In [27]:
#remove non-location informatio and do KMeans clustering, and respond the clustering result to original df
custeringdf = Torontodf.drop(['Postcode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = 6,random_state=0).fit(custeringdf)
Torontodf.insert(0, 'Cluster Labels', kmeans.labels_)
Torontodf.head()

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [30]:
map_toronto_clusters = folium.Map(location=[43.65,-79.35],zoom_start=12)

x = np.arange(6)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, neighbourhood, cluster in zip(Torontodf['Latitude'], Torontodf['Longitude'], Torontodf['Neighbourhood'], Torontodf['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(map_toronto_clusters)
       
map_toronto_clusters

As I guess before, the downtown,Toronto marks are aggregated into a clustering.
All works donw

For the maps not shown here, please acess the 'week2 map.pdf' file on my Github